In [1]:
import dawdreamer
import os
import sys;sys.path.append('../')
import librosa
import torch
import pandas as pd
import matplotlib.pyplot as plt
import math
from src.utils import *
from src.config import ga_settings, daw_settings

SyntaxError: invalid syntax (config.py, line 16)

In [ ]:
# load the preset dataset
df = pd.DataFrame(torch.load('../dataset/processed_preset_dataset_musicnn.pt'))

In [ ]:
# load the target audio (bill evans)
target_audio_path = '../timbre-exp/target-dataset/bill-evans-piano.wav'
target_audio, target_sample_rate = librosa.load(target_audio_path, sr=daw_settings['SAMPLE_RATE'])

# determine the length of the target audio in seconds
target_audio_length = librosa.get_duration(y=target_audio, sr=target_sample_rate)

# obtain the target mfcc
target_mfcc = librosa.feature.mfcc(y=target_audio, sr=target_sample_rate).reshape(-1)

In [ ]:

# find the top 10 presets for the target mfcc
top10presets = find_closest_preset_from_mfcc(target_audio,target_sample_rate,df)

# get closest note from target audio
ff = get_fundamental_frequency(target_audio, target_sample_rate)
target_note = get_closest_note_from_ff(ff[1])

# return the row in the df that corresponds to the top preset
top_preset_row = df[df['preset_names'] == top10presets[0]]

# reset the index of the top_preset_row
top_preset_row = top_preset_row.reset_index(drop=True)

# return the top 10 preset rows in order from the df
top10preset_rows = df[df['preset_names'].isin(top10presets)]

# order the rows based on the order of the top10presets
top10preset_rows = top10preset_rows.set_index('preset_names').loc[top10presets].reset_index()

Likely note: F4, likely frequency: 349.1941058508811
Best Distance Achieved: 546.5276489257812
Likely note: F4, likely frequency: 349.1941058508811


In [ ]:
# show the top 10 presets
top10preset_rows.head()

,preset_names,parameters,parameters_names,mapped_parameter_names,raw_audio,musicnn_features
0,WND Flute 1 FMR,"[0.0, 0.0400000028, 0.288000017, 0.667000055, ...","[@modulation, @dcolfovalue, @dcopwmvalue, @dco...","[{'tal-uno param name': '@modulation', 'dawdre...","{'C2': [tensor(3.4535e-05), tensor(3.5455e-05)...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."
1,ORG Organ 3 FMR,"[0.0, 0.088000007, 1.0, 0.0, 1.0, 0.0, 1.0, 0....","[@modulation, @dcolfovalue, @dcopwmvalue, @dco...","[{'tal-uno param name': '@modulation', 'dawdre...","{'C2': [tensor(-0.0006), tensor(-0.0012), tens...","{'C2': [tensor(-0.0003), tensor(-0.5350), tens..."
2,CHO Voice Chorus FMR,"[0.0, 0.0, 0.5, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...","[@modulation, @dcolfovalue, @dcopwmvalue, @dco...","[{'tal-uno param name': '@modulation', 'dawdre...","{'C2': [tensor(0.0162), tensor(0.0022), tensor...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."
3,BAS Pulse Bass 2 FMR,"[0.0, 0.0, 0.580000043, 0.0, 1.0, 0.0, 0.0, 0....","[@modulation, @dcolfovalue, @dcopwmvalue, @dco...","[{'tal-uno param name': '@modulation', 'dawdre...","{'C2': [tensor(0.0003), tensor(0.0006), tensor...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."
4,WND Flute 2 FMR,"[0.0, 0.0400000028, 0.688000023, 0.667000055, ...","[@modulation, @dcolfovalue, @dcopwmvalue, @dco...","[{'tal-uno param name': '@modulation', 'dawdre...","{'C2': [tensor(-0.0004), tensor(-0.0007), tens...","{'C2': [tensor(-0.0003), tensor(-0.6242), tens..."


In [ ]:
engine = dawdreamer.RenderEngine(sample_rate=target_sample_rate, block_size=daw_settings['BLOCK_SIZE'])
plugin = load_plugin_with_dawdreamer(daw_settings['SYNTH_PLUGIN_PATH_WINDOWS'],daw_settings['SYNTH_NAME'],engine)

In [ ]:
# specify the name of the master parameter you want to create the animation about
master_parameter_name = 'filter env'

In [12]:
# the function below takes the following arguments: top10preset_rows, plugin, engine, target_mfcc, target_audio_length, target_note, ga_settings, daw_settings, print_flag
output = obtain_parameter_objective_space_video_data(plugin,engine,target_mfcc,target_audio_length,target_note,daw_settings,top_preset_row,master_parameter_name,verbosity=1)

# save the output with torch
torch.save(output,'bill-evans-parameter-objective-space-dataset.pt')

Iteration 0 complete for param modulation...
Iteration 1 complete for param dco lfo value...
Iteration 2 complete for param dco pwm value...
Iteration 3 complete for param dco pwm mode...
Iteration 4 complete for param dco pulse enabled...
Iteration 5 complete for param dco saw enabled...
Iteration 6 complete for param dco sub osc enabled...
Iteration 7 complete for param dco sub osc volume...
Iteration 8 complete for param dco noise volume...
Iteration 9 complete for param dco hp filter...
Iteration 10 complete for param filter cutoff...
Iteration 11 complete for param filter resonance...
Iteration 12 complete for param filter env mode...
Iteration 13 complete for param filter env...
Iteration 14 complete for param filter modulation...
Iteration 15 complete for param filter keyboard...
Iteration 16 complete for param master volume...
Iteration 17 complete for param master fine tune...
Iteration 18 complete for param master octave transpose...
Iteration 19 complete for param vca mode..

In [ ]:

# set matplotlib plot settings to use latex font
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
    "font.size": 6
})

# Determine the number of rows and columns for the subplot grid
n = len(output) # number of parameters
cols = math.ceil(math.sqrt(n)) # number of columns
rows = math.ceil(n / cols) # number of rows

fig = plt.figure(figsize=(cols*3, rows*3))

# Define the grid on which to place the subplots
grid = plt.GridSpec(rows, cols, wspace=0.4, hspace=0.4)

# Plot each parameter
for i, data in enumerate(output):
    # Calculate the row and column index for the subplot
    row = i // cols
    col = i % cols

    # Create the subplot on the appropriate part of the grid
    ax = fig.add_subplot(grid[row, col])

    # Plot the data on the subplot
    ax.plot(data['param_values'], data['objective_function_values'])

    # Set the title of the subplot to the parameter name
    ax.set_title(data['param_name'])

    # Set the x and y axis labels
    ax.set_xlabel('Parameter Value')
    ax.set_ylabel('Objective Function Value')

# Show the plot
plt.show()